Pipeline for Extracting Text from Aventura Joven Books

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install tika

  Created wheel for tika: filename=tika-1.24-cp37-none-any.whl size=32885 sha256=71154154143d7e5a5f1eaf7b0d8807367a00efe41ccba55fd3cf1322888da73f
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


In [8]:
files_dir = "/content/drive/MyDrive/capstone/"
filenames = ['Aventura Joven 01 - Persecucion - Elvira Sancho, Jordi Suris.pdf',
              'Aventura Joven 02 - Misterio en - Elvira Sancho, Jordi Suris.pdf',
              'Aventura Joven 03 - Perdidos en - Elvira Sancho, Jordi Suris.pdf',
              'Aventura Joven 04 - La chica de - Elvira Sancho, Jordi Suris.pdf',
              'Aventura Joven 05 - El fantasma - Elvira Sancho, Jordi Suris.pdf',
              'Aventura Joven 06 - El monstruo - Elvira Sancho, Jordi Suris.pdf']

In [9]:
# import parser object from tika
from tika import parser  
import unicodedata
import re

data_list = []
text_orig_list = []
text_proc_list = []
for filename in filenames:
  parsed_pdf = parser.from_file(files_dir+filename) 
  data = parsed_pdf['content'] 
  data_list.append(data)

  # Find beginning and end of text
  beg_idx = data.lower().index('capítulo')
  end_idx = data.lower().index('después de la lectura\n')
  text = data[beg_idx:end_idx]
  text_orig_list.append(text)

  # write the initial text to a file (no preprocessing done here)
  with open(files_dir+filename[:17]+' Text No Preproc.txt', 'w') as f:
    f.write(text)

  # preprocess text
  # remove words with numbers in them, ex altura1 or hostal2 (footnote indicator)
  text_wo_ft_words = text
  for word in re.findall('[-a-zA-ZÀ-ÖØ-öø-ÿ]+»?,?!?\.{0,3}[1234567890]{1,2}', text_wo_ft_words):
    word_idx = text_wo_ft_words.index(word)
    m = re.search('[-a-zA-ZÀ-ÖØ-öø-ÿ]+»?,?!?\.{0,3}', word)
    stripped_word = m.group(0)
    text_wo_ft_words = text_wo_ft_words[:word_idx] + stripped_word + text_wo_ft_words[word_idx+len(word):]

  # remove words that contain '-\n' because they didn't fit on one line
  text_wo_broken_words = text_wo_ft_words
  for word in re.findall('[-a-zA-ZÀ-ÖØ-öø-ÿ]+\-\n{1,2}[-a-zØ-öø-ÿ]+', text_wo_broken_words):
    word_idx = text_wo_broken_words.index(word)
    hyphen_idx = word.index('-')
    if '-\n\n' in word:
      len_sep = 3
    else:
      len_sep = 2
    modified_word = word[:hyphen_idx] + word[hyphen_idx+len_sep:]
    text_wo_broken_words = text_wo_broken_words[:word_idx] + modified_word + text_wo_broken_words[word_idx+len(word):]

  # remove page numbers
  text_wo_page_nums = text_wo_broken_words
  for word in re.findall('\n[0-9]{1,2}\n', text_wo_page_nums):
    word_idx = text_wo_page_nums.index(word)
    m = re.search('[0-9]{1,2}', word)
    text_wo_page_nums = text_wo_page_nums[:word_idx] + '\n' + text_wo_page_nums[word_idx+len(word):]
  
  # remove unnecessary newline breaks
  text_wo_sent_breaks = text_wo_page_nums
  for word in re.findall('[-a-zA-ZÀ-ÖØ-öø-ÿ]+ ?\n\n[-a-zA-ZÀ-ÖØ-öø-ÿ]+', text_wo_sent_breaks):
    word_idx = text_wo_sent_breaks.index(word)
    newline_idx = word.index('\n\n')
    modified_word = word[:newline_idx] + word[newline_idx+2:]
    text_wo_sent_breaks = text_wo_sent_breaks[:word_idx] + modified_word + text_wo_sent_breaks[word_idx+len(word):]
  
  text_proc_list.append(text_wo_sent_breaks)
  
  with open(files_dir+ filename[:17] + ' Text.txt', 'w') as f:
    f.write(text_wo_sent_breaks)

Turning the files into JSON objects

In [59]:
from collections import defaultdict

chapter_indices = defaultdict(str)
data_chap = data_list[0]
print(data_chap.lower().index('capítulo'))
text_len = len(data_list[0])
i = 0
cur_data_chap = 0
while 'capítulo' in data_chap:
  chapter_index = data_chap.lower().index('capítulo')
  newline_after_chapter_index = data_chap[chapter_index+8:].index('\n') + chapter_index+8
  chapter_indices[str(i)] = chapter_index + cur_data_chap
  i += 1
  cur_data_chap += newline_after_chapter_index
  data_chap = data_chap[newline_after_chapter_index:]
  print(len(data_chap))
  

print(chapter_indices)
for chap, chap_idx in chapter_indices.items():
  print(data_list[0][chap_idx: chap_idx+100])
  print("--------------------------------------------------")
  print()

1771
61710
54513
50167
44807
37934
29508
23185
16622
15098
13621
12119
11051
9239
8421
7343
6168
5700
defaultdict(<class 'str'>, {'0': 1771, '1': 8968, '2': 13314, '3': 18674, '4': 25547, '5': 33973, '6': 40296, '7': 46856, '8': 48382, '9': 49860, '10': 51362, '11': 52388, '12': 54242, '13': 55029, '14': 56138, '15': 57313, '16': 57730})
caPítulo 1

—¡Eh, Lucas!, ¡mira, allí!
—¿Dónde?
—Allí. ¿Ves a ese chico?
—¿Quién?
—Aquél, el de los 
--------------------------------------------------

caPítulo 2

Sergio conduce por las rondas de Madrid, moviéndose entre los 
coches. Detrás, un poco l
--------------------------------------------------

caPítulo 3

—¡Hola, primo! —el chico del pelo negro entra en la sala de 
estar y saluda a Sergio.

—
--------------------------------------------------

caPítulo 4

Cuando llegan a la sala del Guernica hay mucha gente mirando 
el cuadro.

—¡Impresionant
--------------------------------------------------

caPítulo �

Sergio vuelve al bar para coger la m

In [ ]:
dict_list = []
author = "Elvira Sancho, Jordi Suris"
for i, (data, text) in enumerate(zip(data_list,text_proc_list)):
  # metadata
  # ISBN
  isbn_index = data_list[0].index('ISBN')
  newline_after_isbn = data_list[0][isbn_index+5:].index('\n\n') + isbn_index+5
  source = data_list[0][isbn_index+6:newline_after_isbn]

  # Level
  if i <= 4:
    level = "A1"
  else:
    level= "A2"

  # Title
  title_index = data_list[0].index('Título')
  newline_after_title = data_list[0][title_index+7:].index('\n\n') + title_index+7
  title = data_list[0][title_index+8:newline_after_title]
  content = text
  # separate by chapters

  # make a dictionary

  dict_list.append(chap_dict)

Attempt at Removing Footnote Definitions

In [ ]:
# def modify_words(regex_find, regex_modify, text, modify):
#   ''' Modifies all words in text that match the regex_find regex to match the regex_modify regex. '''
#   text_removed = text
#   for word in re.findall(regex_find, text_removed):
#     word_idx = text_removed.index(word)
    
#     if modify == 'regex':
#       m = re.search(regex_modify, word)
#       modified_word = m.group(0)
#     elif modify == 'remove':
#       rm_char_idx = word.index(regex_modify)
#       modified_word = word[:rm_char_idx] + word[rm_char_idx+len(regex_modify):]
    
#     text_removed = text_removed[:word_idx] + modified_word + text_removed[word_idx+len(word):]

#   return text_removed

In [ ]:
#text_wo_broken_words = modify_words('[-a-zA-ZÀ-ÖØ-öø-ÿ]+\-\n{1,2}[-a-zØ-öø-ÿ]+', '-\n', text_wo_ft_words, 'remove')
#print(text_wo_broken_words[800:2000])

In [ ]:
# remove the footnotes

In [120]:
# Attempt at removing the footnotes
import re

# normalize the data to ignore special characters in Spanish
#norm_data = unicodedata.normalize('NFD', data).encode('ascii', 'ignore').decode('utf-8')
#norm_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode('utf-8')

# m = re.search('[A-z]+[1234567890]{1,2}', norm_data[1880:])

# m.group(0)
# footnoted_words = [word.strip() for word in re.findall('[A-z]+[1234567890]', norm_data[1880:])]
# footnote_defs = [word.strip() for word in re.findall('[0-9]{1,2}  [A-z]+[\s\w]*:[\w \n]*\.', norm_data[1880:])]

#print(footnoted_words)
#print(footnote_defs)
all_matches = re.findall('([0-9]{1,2}  [A-z]+: ((([A-z]|[0-9]|,)+ ?)+\n))', '''1  altura: Cusco esta a unos 3500 metros de altura, lo que en algunas personas 
provoca el llamado mal de altura o soroche. Los sintomas del mal de altura 
son: dolor de cabeza, mareos, trastornos estomacales y cansancio. Puede com-
batirse con pastillas, ejercicios de respiracion o mate.

2  hostal: alojamiento normalmente mas barato y personal que un hotel, aunque 
sin sus comodidades. La palabra se utiliza principalmente en medios rurales. 
''')
print(len(all_matches))
print(all_matches)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



KeyboardInterrupt: ignored